In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import utils

In [4]:
mnist_folder = '../../mnist'
#utils.download_mnist(mnist_folder)

In [5]:
train, val, test = utils.read_mnist(mnist_folder, flatten=True)
tf.reset_default_graph()

In [6]:
train_data = tf.data.Dataset.from_tensor_slices(train)
test_data = tf.data.Dataset.from_tensor_slices(test)

In [7]:
train_data = train_data.shuffle(10000)
train_data = train_data.batch(128)
test_data = test_data.batch(128)

In [8]:
train_data.output_shapes

(TensorShape([Dimension(None), Dimension(784)]),
 TensorShape([Dimension(None), Dimension(10)]))

In [9]:
iterator = tf.data.Iterator.from_structure(output_shapes=train_data.output_shapes,
                                            output_types=train_data.output_types)
X, Y = iterator.get_next()

In [10]:
tf.shape(X)

<tf.Tensor 'Shape:0' shape=(2,) dtype=int32>

In [11]:
train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

In [12]:
tf.set_random_seed(42)

In [13]:
w = tf.get_variable(initializer=tf.truncated_normal((784,10), dtype=np.float32, mean=0, stddev=0.01), name='weight')
b = tf.get_variable(initializer=tf.zeros((1,10), dtype=np.float32), name='bias')

In [14]:
logits = tf.matmul(X, w) + b

In [102]:
# entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
# loss = tf.reduce_mean(entropy)
loss = tf.losses.hinge_loss(labels=Y, logits=logits)

In [115]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0005).minimize(loss)

In [116]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [117]:
writer = tf.summary.FileWriter('./tensorboard/logreg', tf.get_default_graph())

In [118]:
with tf.Session() as sess:
   
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(30): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/10000))
writer.close()

Average loss epoch 0: 0.2588356328565021
Average loss epoch 1: 0.08295975224396517
Average loss epoch 2: 0.07213086578561816
Average loss epoch 3: 0.06790027028426182
Average loss epoch 4: 0.06534586567864861
Average loss epoch 5: 0.0636243695176618
Average loss epoch 6: 0.06238442861236805
Average loss epoch 7: 0.06137545911898447
Average loss epoch 8: 0.060548923441837
Average loss epoch 9: 0.05983502999987713
Average loss epoch 10: 0.059211713014993554
Average loss epoch 11: 0.058671556313543816
Average loss epoch 12: 0.058178615591727025
Average loss epoch 13: 0.05778217419635418
Average loss epoch 14: 0.057357397684177684
Average loss epoch 15: 0.05705744547466206
Average loss epoch 16: 0.05674909270086954
Average loss epoch 17: 0.05644421839090281
Average loss epoch 18: 0.056207717314015984
Average loss epoch 19: 0.05596365439164084
Average loss epoch 20: 0.055703215158089646
Average loss epoch 21: 0.05553680595097154
Average loss epoch 22: 0.05533529138409121
Average loss epoch 